In [ ]:
# Google Colab has gspread version 3.4.2 by default but we need 4.0.1
!pip install gspread==4.0.1 --quiet
#need to reload runtime, so we crash runtime here and Colab automatically restart it
import os
os.kill(os.getpid(), 9)

In [ ]:
#Import Pandas
import pandas as pd
from IPython.display import display

#Import SKLearn
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

#Other helper packages
import dateutil.parser as dparser
import datetime
import itertools
import os
import random
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import matplotlib.pyplot as plt
import requests

In [ ]:
# Run from here
main()

Hi, I’m Gymmie, your next step gym experience agent!
Please enter your MemberID, or type 'register' if you're a new user: 200
Hello, James! Welcome back. How can I assist you today?

How can I help you today? Please enter 1, 2 or 3 to enter the next menu:

        Option 1 - Visit Planning

        Option 2 - Personal training

        Option 3 - Nutrition and health

        If you wish to exit, enter 'bye'.3

Welcome to your Personal Diet Assistant!

Hello James Jones! I'm your personal dietitionist. Let's create a personalized diet plan for you.

Here is your physical data record from last time:


,Gender,Height,Weight,Index,FirstName,LastName,MemberID,ActivityLevel
199,Male,193,85,5,James,Jones,200,moderately active



Would you like to update your information? (Y/N): N

Here's a personalized meal plan for you:


,Meal,Main Item,Side Item,Beverage,Protein (g),Total fat (g),Total carbohydrate (g)
0,Breakfast,Artisan Egg Sandwich,-,Mocha (L),26.0,10.00,27.00
1,Lunch,Crispy Chicken Sandwich,Ghee Rice with Spicy Chicken,American Mud Pie Shake,20.0,16.00,33.00
2,Dinner,Beef Sandwich,Spicy Chicken,Coke Zero Can,25.0,10.45,56.96



What would you like to do next?
1. Return to personal dietitionist
2. Exit


KeyboardInterrupt: ignored

In [ ]:
def main():
    print("Hi, I’m Gymmie, your next step gym experience agent!")
    member_id = None  # Initialize as None
    global df_gym_database

    # Registration/Login loop
    while member_id is None:
        member_input = input("Please enter your MemberID, or type 'register' if you're a new user: ")

        if member_input.lower() == 'register':
            height = float(input("Please provide your height in cm: "))
            weight = float(input("Please provide your weight in kg: "))
            # Assuming new member IDs are sequential
            new_member_id = df_gym_database["MemberID"].max() + 1
            greeting, df_gym_database = greetUser_updated(new_member_id, df_gym_database, height, weight)
            member_id = new_member_id  # Set the member_id so that the loop exits
        else:
            try:
                member_id = int(member_input)
                greeting, df_gym_database = greetUser_updated(member_id, df_gym_database)
            except ValueError:
                print("Invalid input. Please enter a valid MemberID or type 'register'.")
                continue

        print(greeting)

    # Main interaction loop
    while True:
        user_input = input('''\nHow can I help you today? Please enter 1, 2 or 3 to enter the next menu:\n
        Option 1 - Visit Planning\n
        Option 2 - Personal training\n
        Option 3 - Nutrition and health\n
        If you wish to exit, enter 'bye'.''')

        if 'bye' in user_input.lower():
            print('Stay healthy and see you soon!')
            return
        elif user_input == "1":
            scenario1(member_id)
        elif user_input == "2":
            scenario2(member_id)  # Pass the member_id to scenario2
        elif user_input == "3":
            scenario3(member_id)
        elif 'MNGM' in user_input:  # If the input contains manager identification code, call scenario4()
            scenario4()
        else:
            print("Sorry, I don't understand!\nPlease only enter 1, 2 or 3.")

def greetUser_updated(MemberID, df_gym_database, height=None, weight=None):
    """
    Greet the user based on their MemberID.
    If not in the database and height & weight are provided, register the user and recommend a workout level.
    """
    user_record = df_gym_database[df_gym_database["MemberID"] == MemberID]

    if not user_record.empty:
        user_name = user_record.iloc[0]["FirstName"]
        greeting = f"Hello, {user_name}! Welcome back. How can I assist you today?"
    else:
        if height and weight:
            bmi = calculate_bmi(height, weight)
            bmi_category = categorize_bmi(bmi)
            workout_recommendation = workout_level_recommendation(bmi_category)

            # For this prototype, we'll keep other details as placeholders
            new_user = {
                "Gender": "Unknown",  # Placeholder
                "Height": height,
                "Weight": weight,
                "Index": bmi_category,
                "FirstName": "NewUser",  # Placeholder
                "LastName": "LastName",  # Placeholder
                "MemberID": MemberID
            }
            df_gym_database = df_gym_database.append(new_user, ignore_index=True)

            greeting = f"Thank you for registering! Your MemberID is {MemberID}. How can I assist you today?"
        else:
            greeting = "You're not in our database. Please register by providing your height and weight to get started."

    return greeting, df_gym_database

In [ ]:
#################Helper in multiple scenarios########################
def createDriveURL(url):
  return 'https://drive.google.com/uc?id=' + url.split('/')[-2]

def OpenSheet(url, file_name, sheet_name):
  scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
  credential_url = "https://drive.google.com/file/d/1BVfQsHNoSQfK99u50WI218CjO-_4Uh1s/view?usp=drive_link"
  credentials = requests.get(createDriveURL(credential_url)).json()
  sa = gspread.service_account_from_dict(credentials)
  doc = sa.open(file_name)
  sheet = doc.worksheet(sheet_name)
  return sheet

def isValidGym(user_input):
  gym_info = pd.read_csv(createDriveURL('https://drive.google.com/file/d/1UBMsr5tIGqmUWjk2Q5HhWm8LPYW7uXxX/view?usp=sharing'))
  gym_row = []
  if user_input.isnumeric():
    gym_id = int(user_input)
    gym_row = gym_info[gym_info['gym_id']==gym_id]
  else:
    gym_info['gym_name'] = gym_info['gym_name'].str.lower()
    gym_row = gym_info[gym_info['gym_name']==user_input.lower()]
  if gym_row.empty:
    return 0
  return 1

def continue_or_exit():
    """
    Ask the user if they want to continue, return to the main menu, or exit.
    Returns:
        'continue' to continue with the next step
        'menu' to return to the main menu of the current scenario
        'main' to return to the main menu of the program
        'exit' to exit the program
    """
    while True:
        print("\nWhat would you like to do next?")
        print("1. Return to Previous Menu")
        print("2. Return to Main Menu")
        print("3. Exit")
        choice = input("Enter your choice (1, 2, or 3): ")
        if choice == "1":
            return 'menu'
        elif choice == "2":
            return 'main'
        elif choice == "3":
            return 'exit'
        else:
            print("Invalid choice. Please choose a number between 1 and 3.")

In [ ]:
###############MAIN FUNCTION############################
def scenario1(member_id):

  print('\nOption1 - Ask Gymmie about the gym')
  displayQuotes()
  while True:
      print('Please select an option:')
      print('Option 1 - Ask Gymmie about the traffic')
      print('Option 2 - Get class schedule')
      print('Option 3 - Return to main menu')

      user_input = int(input("Enter your choice (1-3): "))

      if user_input == 3:
        return
      elif user_input == 1:
        gym = input("Please enter the name of your gym or gym id")
        if isValidGym(gym):
          getEstimatedTraffic(gym)
        else:
          print('Please enter a valid gym name or id!')
          print("E.g. 'ActiveLife South' or '1'")
        next_action = continue_or_exit()
      elif user_input == 2:
        gym = input("Please enter the name of your gym or gym id")
        if isValidGym(gym):
          activity = input('Please enter the group class name:')
          getClassSchedule(activity,member_id)
        else:
          print('Please enter a valid gym name or id!')
          print("E.g. 'ActiveLife South' or '1'")
        next_action = continue_or_exit()
      else:
        print('Please choose a valid option!')

      if next_action == 'exit':
          print("Thank you for using the Gym Assistant. Have a great workout!")
          break
      elif next_action == 'main':
          return
      elif next_action == 'menu':
          continue


###############FUNCTIONS USED IN MAIN FUNCTION##########################

def getEstimatedTraffic(gym):
    '''This function takes the date that user requests, parses the input data,
    predict an estimated wait time based on historical data, calculate capacity
    based on prediction and display visit suggestion to users accordingly. '''
    #get the capacity of the gym from google sheet
    capacity = getGymCapacity(gym)
    print("you make ask 'How long is the wait time tonight at 7pm?'")
    print("or 'Is there a lot of people on 2023-08-12 at 9pm?'")
    request = input("Please enter both day and time for accurate suggestion:")
    d, t = parseTime(request)
    num_people = getPrediction(d,t)
    #calculate a percentage of capacity
    capacity_percentage = (num_people/capacity)*100
    print(f'The estimated number of people at that time is {int(num_people)}.')
    print(f'The gym will be at its {float(capacity_percentage):.2f}% capacity.')
    if float(capacity_percentage) > 80:
      print('The gym is crowded now, we suggest you to go at another time')
    elif float(capacity_percentage) > 50:
      print('The gym is start to getting full, we suggest you to visit if you are not in a rush')
    else:
      print('The gym is not busy right now, have a good workout!')
    return

def getGymCapacity(user_input):
  '''read from google form the gym's capacity and returns the number'''
  gym_info = pd.read_csv(createDriveURL('https://drive.google.com/file/d/1UBMsr5tIGqmUWjk2Q5HhWm8LPYW7uXxX/view?usp=sharing'))
  gym_row = []
  #this handles when the gym_id is given
  if user_input.isnumeric():
    gym_id = int(user_input)
    gym_row = gym_info[gym_info['gym_id']==gym_id]
  #this handles when the gym name is given
  else:
    gym_info['gym_name'] = gym_info['gym_name'].str.lower()
    gym_row = gym_info[gym_info['gym_name']==user_input.lower()]
  return gym_row['capacity']

def getClassSchedule(activity,member_id):
  '''This function reads from the csv file where the class schedule is stored.
  Prints out the scheudle time based on suer input.'''
  schedule = pd.read_csv(createDriveURL('https://drive.google.com/file/d/1DCKAXrOFCKgK8xii1fpB9vWwnOP2QYMo/view?usp=sharing'))
  #pre-process data to convert everything to lowercase
  schedule['activity'] = schedule['activity'].str.lower()
  activity = activity.lower()

  #if the schedule that user requested is in the list, display the info
  if not schedule[schedule['activity']==activity].empty:
    year = schedule[schedule['activity']==activity]['year'][0]
    month = schedule[schedule['activity']==activity]['month'][0]
    day = schedule[schedule['activity']==activity]['day'][0]
    hour = schedule[schedule['activity']==activity]['start_hour'][0]
    minute = schedule[schedule['activity']==activity]['start_minute'][0]
    print(f'The next {activity} class is at {hour}:{minute} on {year}-{month}-{day}')

  #if the class that user requested is not on the list, log the request in a form
  else:
    print('Unfortunately, the course is not offered at this gym.')
    print('We have recorded your interest and will carefully consider your inquery.')
    url = 'https://docs.google.com/spreadsheets/d/14coeS-HrWOHkg6KYtss2SyPe2uC7NgxqOMVOp05qV4M/edit?usp=drive_link'
    sheet = OpenSheet(url, "user_data_collection", "inqueries")
    new_row = [str(datetime.date.today()), member_id ,activity]
    sheet.resize(1)
    sheet.append_row(new_row)
  return


###############HELPER FUNCTIONS##########################
def parseTime(string):
  '''Used in getWaitTime to parse date and time strings.
  Returns one standardized date object and one time object.'''
  hour = minute = 0
  # get today's date if user asked for today
  if 'today' in string:
    month = datetime.date.today().date.month
  else:
    #otherwise parse the specified date
    try:
      month = dparser.parse(string,fuzzy=True, dayfirst=True).month
    except (ValueError, OverflowError) as e:
        print('Please enter valid date and time!')
  #get the time now if user asked for now
  if 'now' in string:
    hour = datetime.datetime.now().hour
  else:
    #otherwise get the time specified
    try:
      hour = dparser.parse(string,fuzzy=True, dayfirst=True).hour
    except (ValueError, OverflowError) as e:
        print('Please enter valid date and time!')
  return month,hour

def getPrediction(date, hour):
  '''Used in getWaitTime function to obtain an estimate based on the
  Kaggle Crowded Gym dataset. Returns a boolean for crowdedness.'''
  df = pd.read_csv(createDriveURL("https://drive.google.com/file/d/19PK5jTEXnTOD90W80SGRXuCpO_AqpH8W/view?usp=sharing"))

  X = df[["month","hour"]]
  y = df["number_people"]
  #use random forest regresssion to predict number of people
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
  rf_classifier = RandomForestRegressor(n_estimators=100, random_state=42)
  rf_classifier.fit(X_train, y_train)
  selected_features = ["month", "hour"]
  data = {'month':[int(date)],'hour':[int(hour)]}
  X_test_subset = pd.DataFrame(data, columns=selected_features)
  #use the trained model to get a prediction
  predictions = rf_classifier.predict(X_test_subset)
  #return the result
  return predictions[0]

def displayQuotes():
  '''randomly display a motivational quote in the user prompt'''
  url = "https://drive.google.com/file/d/1n0sr27oKryvbZ4dbR_tYtUqx1tE682N5/view?usp=drive_link"
  quotes = requests.get(createDriveURL(url)).json()['data']
  quote = random.choice(quotes)
  text = quote['quote']
  text_width = 30
  centered_text = text.center(text_width)
  print('              ~~~Daily inspiration~~~                 ')
  print(centered_text)
  print(f"By: {quote['author']}")

In [ ]:
########################################################################################################
#The function takes user height and weight data to match with datasets for recommendations on personal training
########################################################################################################
url1='https://drive.google.com/file/d/1I8iM_NDvXPPxedT5nYKhtRkEPGjsbgok/view?usp=drive_link'
url1='https://drive.google.com/uc?id=' + url1.split('/')[-2]
url2='https://drive.google.com/file/d/1pRiJyKg9ZsGb079pXVzPwTQ42Ya69oOv/view?usp=sharing'
url2='https://drive.google.com/uc?id=' + url2.split('/')[-2]
url3='https://drive.google.com/file/d/1BU14iSgumpaS5SpRJshHZICdsocfSb0t/view?usp=sharing'
url3='https://drive.google.com/uc?id=' + url3.split('/')[-2]
df_gym_database = pd.read_csv(url1)
df_mega_gym = pd.read_csv(url2)
df_mock_training_data=pd.read_csv(url3)
def scenario2(member_id=None):
    """
    Provides an updated interactive interface with an option to update height and weight.
    """

    # Global variables for gym database and training data
    global df_gym_database
    global df_mock_training_data

    # Check if the user is logged in
    if not member_id:
        print("Please register or login first.")
        return

    # Main loop for user interaction
    while True:
        # Displaying the main menu options
        print("\nWelcome to the Personal Training Assistant!")
        print("Please select an option:")
        print("1. View Last Five Sessions")
        print("2. View Favorite Exercises")
        print("3. Mark an Exercise as Favorite")
        print("4. Get Training Recommendations")
        print("5. Get Suggestions for Areas to Train Next")
        print("6. Update Height and Weight")
        print("7. Store a New Training Session")
        print("8. Exit")

        # User input for menu selection
        choice = int(input("Enter your choice (1-8): "))

        # Execute functionality based on user choice
        if choice == 1:
            # View last five training sessions
            sessions = visualize_last_five_sessions(member_id, df_mock_training_data)
            print("\nYour last five sessions are:")
            print(sessions)
            next_action = continue_or_exit()

        elif choice == 2:
            # View favorite exercises
            if member_id:
                favorites = view_favorite_exercises(member_id, df_mock_training_data)
                print("\nYour favorite exercises are:")
                print(favorites)
            else:
                print("Please register or login first using main menu.")
            next_action = continue_or_exit()

        elif choice == 3:
            # Mark an exercise as favorite
            if member_id:
                exercise_name = input("Enter the name of the exercise you'd like to mark as favorite: ")
                df_mock_training_data = mark_favorite(member_id, exercise_name, df_mock_training_data)
                print(f"{exercise_name} has been marked as your favorite.")
            else:
                print("Please register or login first in main menu.")
            next_action = continue_or_exit()

        elif choice == 4:
            # Get training recommendations
            if member_id:
                print("Let's find the best exercises for you!")
                body_focus = input("Do you want to focus on 'upper body', 'lower body', or 'no specific' focus?: ").lower()
                equipment_pref = input("Do you have any equipment preference? For example: 'dumbbells', 'barbell', 'no specific': ").lower()
                user_input = f"{body_focus} with {equipment_pref}"
                recommendations = get_flexible_recommendations(user_input, df_mega_gym, top_n=3)
                print("\nBased on your input, here are some recommended exercises:")
                print(recommendations)
            else:
                print("Please register or login first using main menu.")
            next_action = continue_or_exit()

        elif choice == 5:
            # Get suggestions for body parts to train next
            if member_id:
                suggestions = suggest_missing_body_parts(member_id, df_mock_training_data)
                print("\nBased on your recent sessions, you might want to consider training the following areas:")
                for part, exercise in suggestions.items():
                    print(f"- {part}: Recommended exercise is {exercise}")
            else:
                print("Please register or login first using main menu.")
            next_action = continue_or_exit()

        elif choice == 6:
            # Update height and weight information
            if member_id:
                new_height = float(input("Enter your updated height in cm: "))
                new_weight = float(input("Enter your updated weight in kg: "))
                df_gym_database = update_height_weight(member_id, new_height, new_weight, df_gym_database)
                print("Your height and weight have been updated successfully!")
            else:
                print("Please register or login first using main menu.")
            next_action = continue_or_exit()

        elif choice == 7:
            # Store a new training session
            exercise_name = input("Enter the name of the exercise: ")
            exercise_level = input("Enter the level of the exercise (e.g. beginner, intermediate, advanced): ")
            exercise_body_part = input("Enter the body part targeted by the exercise: ")
            store_training_session(member_id, exercise_name, exercise_level, exercise_body_part)
            print(f"Your training session with exercise '{exercise_name}' has been stored successfully.")
            next_action = continue_or_exit()

        elif choice == 8:
            # Exit the program
            print("Thank you for using the Gym Assistant. Have a great workout!")
            break

        else:
            # Invalid choice
            print("Invalid choice. Please choose a number between 1 and 8.")

        # Check if the user wants to continue, exit, or return to main menu
        if next_action == 'exit':
            print("Thank you for using the Gym Assistant. Have a great workout!")
            break
        elif next_action == 'main':
            return  # This will exit scenario2() and return to the main() menu
        elif next_action == 'menu':
            continue  # Go back to the main menu

def update_height_weight(member_id, new_height, new_weight, df_gym_database):
    """
    Update the height and weight for a given member in the database.
    """
    df_gym_database.loc[df_gym_database["MemberID"] == member_id, "Height"] = new_height
    df_gym_database.loc[df_gym_database["MemberID"] == member_id, "Weight"] = new_weight
    return df_gym_database

def calculate_bmi(height, weight):
    """
    Calculate BMI based on height (in cm) and weight (in kg).
    """
    height_in_m = height / 100  # Convert cm to m
    bmi = weight / (height_in_m ** 2)
    return bmi

def categorize_bmi(bmi):
    """
    Categorize BMI into one of the six categories.
    """
    if bmi < 16:
        return 0  # Extremely Weak
    elif bmi < 16.9:
        return 1  # Weak
    elif bmi < 24.9:
        return 2  # Normal
    elif bmi < 29.9:
        return 3  # Overweight
    elif bmi < 34.9:
        return 4  # Obesity
    else:
        return 5  # Extreme Obesity

def workout_level_recommendation(bmi_category):
    """
    Provide a workout level recommendation based on the BMI category.
    """
    recommendations = {
        0: "beginner level workouts",  # Extremely Weak
        1: "beginner level workouts",  # Weak
        2: "intermediate level workouts",  # Normal
        3: "mixed level workouts",  # Overweight
        4: "mixed level workouts",  # Obesity
        5: "beginner level workouts with a focus on low impact"  # Extreme Obesity
    }
    return recommendations.get(bmi_category, "mixed level workouts")



def get_flexible_recommendations(user_input, df_mega_gym, top_n=3):
    """
    Get exercise recommendations based on the user's input, with flexibility.
    """
    # Check if expected columns exist in the DataFrame
    required_columns = ["Title", "Desc", "BodyPart"]
    for column in required_columns:
        if column not in df_mega_gym.columns:
            return f"Error: Expected column '{column}' is missing in the dataset."

    # Handle non-specific inputs
    if "no specific" in user_input.lower():
        return df_mega_gym.sample(top_n)[["Title", "Type", "BodyPart", "Equipment", "Rating"]]

    # Handle specific inputs with flexibility
    user_input_parts = user_input.lower().split()
    if "upper body" in user_input.lower():
        upper_body_parts = ["chest", "shoulders", "triceps", "biceps", "back"]
        df_mega_gym["Priority"] = df_mega_gym["BodyPart"].str.lower().isin(upper_body_parts).astype(int)
    elif "lower body" in user_input.lower():
        lower_body_parts = ["quadriceps", "hamstrings", "glutes", "calves"]
        df_mega_gym["Priority"] = df_mega_gym["BodyPart"].str.lower().isin(lower_body_parts).astype(int)
    else:
        df_mega_gym["Priority"] = 0

    # TF-IDF and cosine similarity calculations
    combined_data = df_mega_gym["Title"].astype(str) + " " + df_mega_gym["Desc"].astype(str)
    combined_data = combined_data.append(pd.Series(user_input), ignore_index=True)
    tfidf_vectorizer = TfidfVectorizer(stop_words='english')
    tfidf_matrix = tfidf_vectorizer.fit_transform(combined_data)
    cosine_similarities = linear_kernel(tfidf_matrix[-1], tfidf_matrix[:-1]).flatten()

    # Add priority to cosine similarity to give preference to relevant body parts
    cosine_similarities += df_mega_gym["Priority"].values

    # Get top recommendations
    recommended_indices = cosine_similarities.argsort()[-top_n:][::-1]
    recommended_exercises = df_mega_gym.iloc[recommended_indices]
    return recommended_exercises[["Title", "Type", "BodyPart", "Equipment", "Rating"]]

def get_workout_description_updated(workout_title, df_mega_gym):
    """
    Retrieve the description of a specific workout based on its title.
    If the description is missing, provide an appropriate message.
    """
    workout_info = df_mega_gym[df_mega_gym["Title"] == workout_title]

    if not workout_info.empty:
        description = workout_info.iloc[0]["Desc"]
        if pd.isna(description):
            return "Sorry! This workout does not have a description."
        else:
            return description
    else:
        return "Sorry, I couldn't find information on that workout."


########################################################################################################
#Below serves to memorized user's perference
########################################################################################################

def store_training_session(member_id, exercise, level, body_part, df_training_data):
    """
    Store a user's training session.
    """
    new_session = {
        "MemberID": member_id,
        "Exercise": exercise,
        "Level": level,
        "BodyPart": body_part,
        "Favorite": False  # Default value when a new session is added
    }
    df_training_data = df_training_data.append(new_session, ignore_index=True)
    return df_training_data

def view_favorite_exercises(member_id, df_training_data):
    """
    View the favorite exercises for a given user.
    """
    return df_training_data[(df_training_data["MemberID"] == member_id) & (df_training_data["Favorite"] == True)]

def mark_favorite(member_id, exercise, df_training_data):
    """
    Mark an exercise as a favorite for a given user.
    """
    df_training_data.loc[(df_training_data["MemberID"] == member_id) & (df_training_data["Exercise"] == exercise), "Favorite"] = True
    return df_training_data

def visualize_last_five_sessions(member_id, df_training_data):
    """
    Display the last five sessions for a given user.
    """
    user_sessions = df_training_data[df_training_data["MemberID"] == member_id]
    return user_sessions.tail(5)

def suggest_missing_body_parts(member_id, df_training_data):
    """
    Suggest exercises for body parts that the user hasn't trained in their last five sessions.
    """
    # Get the body parts from the last five sessions
    last_five_sessions = visualize_last_five_sessions(member_id, df_training_data)
    trained_body_parts = last_five_sessions["BodyPart"].unique()

    # Get all possible body parts
    all_body_parts = df_training_data["BodyPart"].unique()

    # Identify missing body parts
    missing_body_parts = set(all_body_parts) - set(trained_body_parts)

    # Generate suggestions based on missing body parts
    suggestions = {}
    for body_part in missing_body_parts:
        exercises = df_mega_gym[df_mega_gym["BodyPart"] == body_part]["Title"].sample(1).tolist()
        if exercises:
            suggestions[body_part] = exercises[0]

    return suggestions

In [ ]:
#######################################################################################################################################
#The function takes user height, weight, activity level data to match with datasets for recommendations on meal plan
#######################################################################################################################################

def scenario3(member_id):

    # Extract data
    url1='https://drive.google.com/file/d/1iNUMp8k7yE2Ych516AfXcy8XCv-ec9GR/view?usp=sharing'
    url1='https://drive.google.com/uc?id=' + url1.split('/')[-2]
    url2='https://drive.google.com/file/d/1wG8CgKRmENYqlsLfA1zlIkqs2EREpFpx/view?usp=sharing'
    url2='https://drive.google.com/uc?id=' + url2.split('/')[-2]
    df_gym_database = pd.read_csv(url1)
    df_meals = pd.read_csv(url2)

    while True:
      print("\nWelcome to your Personal Diet Assistant!")
      #user_input = input("Please enter your MemberID: ")
      #MemberID = int(user_input)
      MemberID = member_id
      greetUser(MemberID, df_gym_database)
      next_action = continue_or_exit()
      if next_action == 'exit':
          print("Thank you for using the Gym Assistant. Enjoy the meal!")
          print()
          break
      elif next_action == 'menu':
          continue

##################################################################
# Helper Functions
##################################################################

# Greet User, check if have registered with body measurement data
def greetUser(MemberID, df_gym_database):
    Member_data = df_gym_database[df_gym_database['MemberID'] == MemberID]

    if not Member_data.empty:
        first_name = Member_data['FirstName'].values[0]
        last_name = Member_data['LastName'].values[0]
        print()
        print(f"Hello {first_name} {last_name}! I'm your personal dietitionist. Let's create a personalized diet plan for you.")
        print()
        print("Here is your physical data record from last time:")
        display(Member_data)
        print()
        user_input = input("Would you like to update your information? (Y/N): ")
        if user_input == "Y":
            updateUserData(MemberID, df_gym_database)
        else:
            suggest_meal_plan(MemberID, df_gym_database)
    else:
        updateUserData(MemberID, df_gym_database)

# Function for update user physical data in the gym database
def updateUserData(MemberID, df_gym_database):
    url1='https://drive.google.com/file/d/1iNUMp8k7yE2Ych516AfXcy8XCv-ec9GR/view?usp=sharing'
    url1='https://drive.google.com/uc?id=' + url1.split('/')[-2]

    weight = float(input("Enter your weight (kg): "))
    activity_level = input("How active are you (sedentary, lightly active, moderately active, very active)? ").lower()

    # Update DataFrame with new data
    df_gym_database.loc[df_gym_database['MemberID'] == MemberID, 'Weight'] = weight
    df_gym_database.loc[df_gym_database['MemberID'] == MemberID, 'ActivityLevel'] = activity_level

    # Save updated DataFrame to CSV
    df_gym_database.to_csv(url1, index=False)

    suggest_meal_plan(MemberID, df_gym_database)


# Function for suggest meal plan
def suggest_meal_plan(MemberID, df_gym_database):
    url2='https://drive.google.com/file/d/1wG8CgKRmENYqlsLfA1zlIkqs2EREpFpx/view?usp=sharing'
    url2='https://drive.google.com/uc?id=' + url2.split('/')[-2]
    df_meals = pd.read_csv(url2)

    # Calculate daily calorie requirement
    weight = df_gym_database[df_gym_database['MemberID'] == MemberID]['Weight'].values[0]
    height = df_gym_database[df_gym_database['MemberID'] == MemberID]['Height'].values[0]
    gender = df_gym_database[df_gym_database['MemberID'] == MemberID]['Gender'].values[0]

    if gender == "Male":
        BMR = 88.362 + (13.397 * weight) + (4.799 * height) - (5.677 * 30)  # Placeholder for age
    else:
        BMR = 447.593 + (9.247 * weight) + (3.098 * height) - (4.330 * 30)

    activity_multipliers = {
        "sedentary": 1.2,
        "lightly active": 1.375,
        "moderately active": 1.55,
        "very active": 1.725
    }

    activity_level = df_gym_database[df_gym_database['MemberID'] == MemberID]['ActivityLevel'].values[0]
    calories_needed = BMR * activity_multipliers.get(activity_level, 1.2)

    # Calculate calories for each meal
    breakfast_calories = int(calories_needed * 0.3)
    lunch_calories = int(calories_needed * 0.5)
    dinner_calories = int(calories_needed * 0.2)

    tolerance = 150

    breakfast_options = df_meals[df_meals['Menu Category'].isin(['breakfast_main', 'beverage_morning'])]
    lunch_options = df_meals[df_meals['Menu Category'].isin(['meal_main_lunch', 'meal_side', 'beverage'])]
    dinner_options = df_meals[df_meals['Menu Category'].isin(['meal_main', 'meal_side', 'beverage'])]

    # Get meal combinations and find matching meal plans
    breakfast_combinations = list(itertools.product(
        breakfast_options[breakfast_options['Menu Category'] == 'breakfast_main'].iterrows(),
        breakfast_options[breakfast_options['Menu Category'] == 'beverage_morning'].iterrows()
    ))

    lunch_combinations = list(itertools.product(
        lunch_options[lunch_options['Menu Category'] == 'meal_main_lunch'].iterrows(),
        lunch_options[lunch_options['Menu Category'] == 'meal_side'].iterrows(),
        lunch_options[lunch_options['Menu Category'] == 'beverage'].iterrows()
    ))

    dinner_combinations = list(itertools.product(
        dinner_options[dinner_options['Menu Category'] == 'meal_main'].iterrows(),
        dinner_options[dinner_options['Menu Category'] == 'meal_side'].iterrows(),
        dinner_options[dinner_options['Menu Category'] == 'beverage'].iterrows()
    ))

    # Finding meals matching the desired calorie count
    breakfast_meal = next((combo for combo in breakfast_combinations if (combo[0][1]['Energy (kCal)'] + combo[1][1]['Energy (kCal)'] >= breakfast_calories - tolerance) and (combo[0][1]['Energy (kCal)'] + combo[1][1]['Energy (kCal)'] <= breakfast_calories + tolerance)), None)
    lunch_meal = next((combo for combo in lunch_combinations if (combo[0][1]['Energy (kCal)'] + combo[1][1]['Energy (kCal)'] + combo[2][1]['Energy (kCal)'] >= lunch_calories - tolerance) and (combo[0][1]['Energy (kCal)'] + combo[1][1]['Energy (kCal)'] + combo[2][1]['Energy (kCal)'] <= lunch_calories + tolerance)), None)
    dinner_meal = next((combo for combo in dinner_combinations if (combo[0][1]['Energy (kCal)'] + combo[1][1]['Energy (kCal)'] + combo[2][1]['Energy (kCal)'] >= dinner_calories - tolerance) and (combo[0][1]['Energy (kCal)'] + combo[1][1]['Energy (kCal)'] + combo[2][1]['Energy (kCal)'] <= dinner_calories + tolerance)), None)

    # Displaying meal plan
    if breakfast_meal and lunch_meal and dinner_meal:
        print("\nHere's a personalized meal plan for you:")

        # Can extend the data for display by un-comment the respective rows
        data = {
        "Meal": ["Breakfast", "Lunch", "Dinner"],
        "Main Item": [breakfast_meal[0][1]['Menu Items'], lunch_meal[0][1]['Menu Items'], dinner_meal[0][1]['Menu Items']],
        "Side Item": ["-", lunch_meal[1][1]['Menu Items'], dinner_meal[1][1]['Menu Items']],
        "Beverage": [breakfast_meal[1][1]['Menu Items'], lunch_meal[2][1]['Menu Items'], dinner_meal[2][1]['Menu Items']],
        "Protein (g)": [breakfast_meal[0][1]['Protein (g)'], lunch_meal[0][1]['Protein (g)'], dinner_meal[0][1]['Protein (g)']],
        "Total fat (g)": [breakfast_meal[0][1]['Total fat (g)'], lunch_meal[0][1]['Total fat (g)'], dinner_meal[0][1]['Total fat (g)']],
        #"Sat Fat (g)": [breakfast_meal[0][1]['Sat Fat (g)'], lunch_meal[0][1]['Sat Fat (g)'], dinner_meal[0][1]['Sat Fat (g)']],
        #"Trans fat (g)": [breakfast_meal[0][1]['Trans fat (g)'], lunch_meal[0][1]['Trans fat (g)'], dinner_meal[0][1]['Trans fat (g)']],
        #"Cholesterols (mg)": [breakfast_meal[0][1]['Cholesterols (mg)'], lunch_meal[0][1]['Cholesterols (mg)'], dinner_meal[0][1]['Cholesterols (mg)']],
        "Total carbohydrate (g)": [breakfast_meal[0][1]['Total carbohydrate (g)'], lunch_meal[0][1]['Total carbohydrate (g)'], dinner_meal[0][1]['Total carbohydrate (g)']],
        #"Total Sugars (g)": [breakfast_meal[0][1]['Total Sugars (g)'], lunch_meal[0][1]['Total Sugars (g)'], dinner_meal[0][1]['Total Sugars (g)']],
        #"Sodium (mg)": [breakfast_meal[0][1]['Sodium (mg)'], lunch_meal[0][1]['Sodium (mg)'], dinner_meal[0][1]['Sodium (mg)']]
          }

        meal_plan_df = pd.DataFrame(data)
        display(meal_plan_df)
    else:
        print("\nSorry, we would need to improve our meal plan database to provide the meal plan for your calories range.")

# The function that gives user options to return to the scenario's sub menu for the main menu
def continue_or_exit():
    """
    Ask the user if they want to continue, return to the main menu, or exit.
    Returns:
        'continue' to continue with the next step
        'menu' to return to the main menu
        'exit' to exit the program
    """
    while True:
        print("\nWhat would you like to do next?")
        print("1. Return to personal dietitionist")
        print("2. Exit")
        #maybe link to another section in main menu
        choice = input("Enter your choice (1 or 2): ")
        if choice == "1":
            return 'menu'
        elif choice == "2":
            return 'exit'
        else:
            print("Invalid choice. Please choose a number between 1 and 2.")


In [ ]:
def scenario4():
  # Display the main menu with options for the operation dashboard
  while True:
    print('\nOption4 - Welcome to the operation dashboard. Please select an option from the menu:')
    print('1 - Track equipment attrition')
    print('2 - View insights on the new classes demande')
    print('3 - Generate report for gym operation business performance')
    print('4 - Go back to main menu')

    # get user input for menu selection
    user_input = int(input("Enter your choice (1-4): "))

    if user_input == 4:
          return

    if user_input == 1:
      # Option to track equipment attrition
      gym_id = input('Please enter the gym id: ')
      if isValidGym(gym_id):
        mintues = 500
        equipmetns_to_maintain = ", ".join(getListToMaintain(int(gym_id), mintues))
        print(f'The total usage time of {equipmetns_to_maintain} equipment has surpassed {mintues} minutes.')
        print('Please provide maintenance on them.')
      else:
        print('Please enter a valid gym id!')
      next_action = continue_or_exit()

    if user_input == 2:
      # Option to view insights on new class demand
      getFreqClass()
      next_action = continue_or_exit()

    if user_input == 3:
      # Option to go to the report menu for gym operation business performance
      getReport()
      next_action = continue_or_exit()


    if next_action == 'exit':
        print("Thank you for using the Gym Assistant. Have a great workout!")
        break
    elif next_action == 'main':
        return
    elif next_action == 'menu':
        continue

###############FUNCTIONS USED IN MAIN FUNCTION##########################
def getListToMaintain(gym_id, threshold_minutes):
  '''Calcualtes the total number of minutes used per equipment from the equipment
   tracking dataset, based on the input number will return a list of equipmetns
   that requires maintenance.'''
  attrition = pd.read_csv(createDriveURL("https://drive.google.com/file/d/1GtxrB6F_ZXaq4AoHbU1rjPA5NEG4mKcb/view?usp=drive_link"))
  attrition["date_of_usage"] = pd.to_datetime(attrition["date_of_usage"])
  current_month = pd.Timestamp.today().replace(day=1)
  attrition = attrition[attrition["date_of_usage"].dt.month == current_month.month]
  attrition = attrition[attrition['gym_id'] == gym_id]
  grouped_attrition = attrition.groupby(["equipment_name"])["duration_minutes"].sum().reset_index()
  maintenance = grouped_attrition[grouped_attrition['duration_minutes']>threshold_minutes]
  equipment_to_maintain = maintenance['equipment_name'].to_string(header=False, index=False).replace(' ','').split('\n')
  return equipment_to_maintain

def getFreqEquipment():
  '''Get the most popular equipment of the current month.'''
  attrition = pd.read_csv(createDriveURL("https://drive.google.com/file/d/1GtxrB6F_ZXaq4AoHbU1rjPA5NEG4mKcb/view?usp=drive_link"))
  current_month = pd.Timestamp.today().replace(day=1)
  attrition["date_of_usage"] = pd.to_datetime(attrition["date_of_usage"])
  current_month_data = attrition[attrition["date_of_usage"].dt.month == current_month.month]
  equipment_usage_count = current_month_data["equipment_name"].value_counts().reset_index()
  equipment_usage_count.columns = ["equipment_name", "Usage Count"]
  top_equipment = equipment_usage_count.nlargest(1, "Usage Count")['equipmentm_name'][0]
  return top_equipment

def getFreqClass():
  '''Return a list of most mentioned classes that they already offer and don’t provide yet based on user input .'''
  url = 'https://docs.google.com/spreadsheets/d/14coeS-HrWOHkg6KYtss2SyPe2uC7NgxqOMVOp05qV4M/edit?usp=drive_link'
  sheet = OpenSheet(url, "user_data_collection", "inqueries")
  df = pd.DataFrame(sheet.get_all_records())
  activity_counts = df['activity'].value_counts()
  print('Here is a list of most request activities of the current month:')
  display(activity_counts)

def getReport():
  '''Analytics feature.
  Generates a weekly report including number of people visited, total number of members, new members who joined this week, and more.'''

  print('Please choose a form or plot to review')
  while True:
    print('1 - top n frequently visited members')
    print('2 - members who did not visit for x days')
    print('3 - equipment usage plot')
    print('4 - monthly user count')
    print('5 - Go back to previosu menu')

    user_input = int(input("Enter your choice (1-5): "))
    next_action = None

    if user_input == 5:
      return

    if user_input == 1:
      n = int(input('Specify the number of users you want to see'))
      top_n_frequent_members(n)
      next_action = continue_or_exit()

    elif user_input == 2:
      x = int(input('Specify x to see members who did not visit for x days'))
      getLeavingMember(x)
      next_action = continue_or_exit()

    elif user_input == 3:
      id = int(input('Splecify the gym id you want to see'))
      plotEquipementUsage(id)
      next_action = continue_or_exit()

    elif user_input == 4:
      plotMonthlyUserCount()
      next_action = continue_or_exit()

    else:
      print('Please enter a valid option!')

    if next_action == 'exit':
        print("Thank you for using the Gymmie Assistant. Have a good day!")
        break
    elif next_action == 'main':
        return
    elif next_action == 'menu':
        continue

  return

###########################################################################################
def top_n_frequent_members(n):
    '''Show a list of users who frequently visited the gym in the current month'''
    df = pd.read_csv(createDriveURL("https://drive.google.com/file/d/1GtxrB6F_ZXaq4AoHbU1rjPA5NEG4mKcb/view?usp=drive_link"))
    df2 = pd.read_csv(createDriveURL("https://drive.google.com/file/d/1iNUMp8k7yE2Ych516AfXcy8XCv-ec9GR/view?usp=drive_link"))

    # filter data for the current month
    current_month = datetime.datetime.now().month
    df['date_of_usage'] = pd.to_datetime(df['date_of_usage'])
    df = df[df['date_of_usage'].dt.month == current_month]

    # Group data by gym_id and user_number and count visits
    grouped = df.groupby(['gym_id', 'user_number']).size().reset_index(name='visit_count')

    # Sort the data by gym_id and visit_count
    sorted_data = grouped.sort_values(['gym_id', 'visit_count'], ascending=[True, False])

    # Get the top n most frequent members for each gym
    top_n_members = sorted_data.groupby('gym_id').head(n)
    top_n_members = top_n_members[['gym_id', 'user_number', 'visit_count']]

    result_df = top_n_members.merge(df2, left_on='user_number', right_on='MemberID', how='inner')
    display(result_df)


def getLeavingMember(days):
  '''Show a list of users who have not visited the gym for given number of days'''
  df = pd.read_csv(createDriveURL("https://drive.google.com/file/d/1GtxrB6F_ZXaq4AoHbU1rjPA5NEG4mKcb/view?usp=drive_link"))
  df2 = pd.read_csv(createDriveURL("https://drive.google.com/file/d/1iNUMp8k7yE2Ych516AfXcy8XCv-ec9GR/view?usp=drive_link"))

  # Convert 'Date of Usage' to a datetime object
  df['date_of_usage'] = pd.to_datetime(df['date_of_usage'])

  # Calculate the current date, plus 40 for the purpose of demo
  current_date = datetime.datetime.now()+datetime.timedelta(40)

  # Calculate the last visit date for each member
  last_visit_date = df.groupby('user_number')['date_of_usage'].max().reset_index()

  # Identify members whose last visit date is more than x days ago
  members_not_visited = last_visit_date[(current_date - last_visit_date['date_of_usage']) > datetime.timedelta(days=days)]

  result_df = members_not_visited.merge(df2, left_on='user_number', right_on='MemberID', how='inner')
  display(result_df)


def plotEquipementUsage(id=0):
  '''get equipment usage of the current month'''
  #get date and filter by gym
  equipment_data = pd.read_csv(createDriveURL("https://drive.google.com/file/d/1GtxrB6F_ZXaq4AoHbU1rjPA5NEG4mKcb/view?usp=drive_link"))

  #if the gym_id is specified, we return the result for specific gym
  #otherwise we return for all
  if id !=0:
    equipment_data = equipment_data[equipment_data['gym_id'] == id]

  # Convert 'date_of_usage' string to datetime
  equipment_data['date_of_usage'] = pd.to_datetime(equipment_data['date_of_usage'])

  # filter data for the current month
  current_month = datetime.datetime.now().month
  equipment_data_current_month = equipment_data[equipment_data['date_of_usage'].dt.month == current_month]

  # count equipment usage by name
  usage_counts = equipment_data_current_month['equipment_name'].value_counts()

  # create a bar plot
  plt.figure(figsize=(10, 6))
  usage_counts.plot(kind='bar', color='skyblue')
  plt.title('Equipment Usage for Current Month')
  plt.xlabel('Equipment Name')
  plt.ylabel('Usage Count')
  plt.xticks(rotation=45)
  plt.tight_layout()

  # show the plot
  plt.show()

def plotMonthlyUserCount():
  '''Plot the number of users of teh gyms of the current month'''
  df = pd.read_csv(createDriveURL("https://drive.google.com/file/d/1GtxrB6F_ZXaq4AoHbU1rjPA5NEG4mKcb/view?usp=drive_link"))

  # convert date string to datetime
  df['date_of_usage'] = pd.to_datetime(df['date_of_usage'])

  # get the current year and month
  current_year = datetime.datetime.now().year
  current_month = datetime.datetime.now().month

  #filter data for the current year and month
  df = df[(df['date_of_usage'].dt.year == current_year) & (df['date_of_usage'].dt.month == current_month)]

  #group data by date and count the number of users per day
  daily_average_users = df.groupby(df['date_of_usage'].dt.date)['user_number'].count()

  #create a continuous line plot
  plt.figure(figsize=(10, 6))
  plt.plot(daily_average_users, marker='o', linestyle='-')
  plt.title(f'Number of Users per Day in Current Month ({datetime.datetime.now().strftime("%B %Y")})')
  plt.xlabel('Date')
  plt.ylabel('Average Users')
  plt.xticks(rotation=45)
  plt.grid(True)
  plt.tight_layout()
  plt.show()